## Train

Keras-RetinaNet 모델 훈련 과정입니다. Keras 2.1.2, [Keras-RetinaNet](https://github.com/fizyr/keras-retinanet), [Keras-ResNet](https://github.com/broadinstitute/keras-resnet) 패키지가 필요합니다. 다음은 환경 구축에 필요한 사전 과정입니다.

 - keras>=2.1.2가 설치되어있는지 재확인합니다.
 ```
 pip install keras --upgrade
 ```
 - 위에서 명시한 각 패키지를 다운로드 및 설치합니다.
 ```
 git clone https://github.com/fizyr/keras-retinanet.git
 cd keras-retinanet
 python setup.py install
 pip install --upgrade git+https://github.com/broadinstitute/keras-resnet
 ```

In [ ]:
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input
from keras.optimizers import adam
from keras.preprocessing.image import ImageDataGenerator
from keras_retinanet.losses import focal, smooth_l1
from keras_retinanet.models.resnet import ResNet50RetinaNet
from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator

import tensorflow as tf

### 데이터셋 설정

아래는 데이터셋 및 훈련 설정에 필요한 환경 변수입니다. 사용하는 데이터셋 및 모델에 맞춰서 변경하시기 바랍니다.

 - `classes` : 데이터내에 구성하는 모든 클래스의 정보를 담은 dict입니다. 키는 클래스 이름, 값은 각 클래스의 일련번호로 지정해야 합니다. 일련번호는 숫자 0부터 중복과 누락없이 채워져야 합니다. `None` 지정시 Pascal VOC의 기본값이 됩니다.
 - `dataset_path` : 데이터셋이 존재하는 경로를 지정합니다.
 - `train_dataset_name` : 데이터셋 내에서 훈련용으로 사용할 ImageSets 묶음의 이름입니다.
 - `val_dataset_name` : 데이터셋 내에서 검증용으로 사용할 ImageSets 묶음의 이름입니다.
 - `weight_file_path` : 결과 weight파일이 출력될 경로 및 이름을 지정합니다.
 - `epochs` : 총 몇 epoch를 실행할 건지를 지정할 수 있습니다.
 - `epoch_len` : 한 epoch당 몇 샘플을 훈련할 것인지를 지정할 수 있습니다. 이 값이 1보다 작으면 한 epoch마다 훈련용 데이터셋 전체를 순회합니다.
 - `batch_size` : 한 훈련에 몇개의 샘플을 일괄 훈련할 것인지를 지정합니다. 훈련 도중 OOM 오류가 발생하면 이 값을 낮춰서 재시도하시기 바랍니다.
 - `earlystopping` : 학습 조기종료(EarlyStopping)의 patience를 지정합니다. 이 epoch 횟수동안 val_loss에 진전이 없으면 학습이 종료됩니다. 20 이상을 권장합니다.

In [ ]:
classes = None
dataset_path = './dataset/PascalVOC2012'
train_dataset_name = 'train'
val_dataset_name = 'val'
weight_file_path = './retinanet.h5'
epochs = 2000
epoch_len = 1000
batch_size = 4
earlystopping = 20

if classes is None:
    classes = {'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4,
               'bus': 5, 'car': 6, 'cat': 7, 'chair': 8, 'cow': 9,
               'diningtable': 10, 'dog': 11, 'horse': 12, 'motorbike': 13, 'person': 14,
               'pottedplant': 15, 'sheep': 16, 'sofa': 17, 'train': 18, 'tvmonitor': 19}

### 모델 빌드

RetinaNet with ResNet50 모델을 빌드합니다.

In [ ]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

check_keras_version()

set_session(get_session())

image_input = Input((None, None, 3))
model = ResNet50RetinaNet(image_input, num_classes=len(classes), weights='imagenet', nms=False)

model.compile(
    loss={'regression': smooth_l1(), 'classification': focal()},
    optimizer=adam(lr=1e-5, clipnorm=0.001)
)

### 데이터셋 로딩

훈련 및 검증에 필요한 데이터셋을 로드합니다.

In [ ]:
train_image_data_generator = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True
)
val_image_data_generator = ImageDataGenerator()

train_generator = PascalVocGenerator(
    dataset_path,
    train_dataset_name,
    train_image_data_generator,
    batch_size = batch_size,
    classes = classes
)

val_generator = PascalVocGenerator(
    dataset_path,
    val_dataset_name,
    val_image_data_generator,
    batch_size = batch_size,
    classes = classes
)

### 훈련 시작

데이터셋에 맞게 모델의 훈련을 시작합니다. 훈련 과정에서 val_loss가 최저인 weight가 자동으로 저장됩니다.

In [ ]:
if epoch_len < 1:
    epoch_len = len(train_generator.image_names)

model.fit_generator(
    generator = train_generator,
    steps_per_epoch = epoch_len // batch_size,
    epochs = epochs,
    verbose = 2,
    validation_data = val_generator,
    validation_steps = 200 // batch_size,
    callbacks = [
        ModelCheckpoint(weight_file_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
        EarlyStopping(monitor='val_loss', min_delta=0, patience=earlystopping, verbose=1, mode='auto')
    ]
)